In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
from datasets import load_dataset
from transformers import RobertaForSequenceClassification,  Trainer, TrainingArguments
from transformers import RobertaTokenizer
from transformers import BertForSequenceClassification, BertTokenizer
import torch
import numpy as np
from sklearn.metrics import accuracy_score
from transformers import DataCollatorWithPadding

In [4]:
from transformers import RobertaTokenizerFast
import pandas as pd
from datasets import Dataset, DatasetDict
splits = {'train': 'plain_text/train-00000-of-00001.parquet', 'test': 'plain_text/test-00000-of-00001.parquet', 'unsupervised': 'plain_text/unsupervised-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/stanfordnlp/imdb/" + splits["train"])
dataset = Dataset.from_pandas(df)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
print(dataset["label"].count(1))
dataset = dataset.train_test_split(test_size=0.2)

12500


In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [7]:
roberta_model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def train_model(model, tokenized_data, model_name, data_collator):
    training_args = TrainingArguments(
        output_dir=f"./{model_name}-imdb",
        per_device_train_batch_size=16,  # Optimized for T4
        per_device_eval_batch_size=32,
        eval_strategy="epoch",           # Changed from evaluation_strategy
        save_strategy="epoch",
        logging_strategy="epoch",        # Changed from logging_steps
        fp16=True,                      # Enable mixed-precision
        learning_rate=2e-5,
        num_train_epochs=2,             # Quick test run
        weight_decay=0.01,
        gradient_accumulation_steps=2,  # Helps with T4's VRAM limits
        report_to="none",               # Disable WandB logging
        dataloader_num_workers=2,       # Faster data loading
        remove_unused_columns=False      # Keep label column for loss calculation
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_data["train"],
        eval_dataset=tokenized_data["test"],
        compute_metrics=lambda eval_pred: {
            "accuracy": accuracy_score(eval_pred.label_ids, np.argmax(eval_pred.predictions, axis=1))
        },
        data_collator=data_collator, # Add data collator here
    )
    trainer.train()
    return trainer.evaluate()

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:

print("\nTraining RoBERTa...")
roberta_results = train_model(roberta_model, tokenized_dataset, "roberta",data_collator=data_collator)



Training RoBERTa...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.122500,0.164155,0.944800


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [8]:
collator=DataCollatorWithPadding(tokenizer=bert_tokenizer)

In [9]:
def correct_tokenize(examples):
    return bert_tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors="pt"
    )


bert_tokenized_dataset = dataset.map(
    correct_tokenize,
    batched=True,
    remove_columns=["text"]
)

def train_bert(model, dataset, collator):
    training_args = TrainingArguments(
        output_dir="./bert-finetuned",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        eval_strategy="steps",
        eval_steps=100,
        logging_steps=10,
        fp16=True,
        dataloader_num_workers=2,
        remove_unused_columns=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=bert_tokenized_dataset["train"],
        eval_dataset=bert_tokenized_dataset["test"],
        data_collator=collator,
        compute_metrics=lambda eval_pred: {
            "accuracy": accuracy_score(eval_pred.label_ids, np.argmax(eval_pred.predictions, axis=1))
        },

    )

    # Critical: Verify one batch works
    try:
        trainer.evaluate()  # Test evaluation first
        print("✅ Evaluation successful, starting training...")
        trainer.train()
    except Exception as e:
        print(f"❌ Failed: {str(e)}")
        raise

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [11]:

print("\nTraining BERT...")
bert_results = train_bert(bert_model, bert_tokenized_dataset,collator=collator)


Training BERT...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shahmeeramir (shahmeeramir-nuces) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ Evaluation successful, starting training...


Step,Training Loss,Validation Loss,Model Preparation Time,Accuracy
100,0.428200,0.505132,0.003300,0.787800
200,0.423800,0.468578,0.003300,0.802800
300,0.597000,0.469687,0.003300,0.845200
400,0.389500,0.429067,0.003300,0.868000
500,0.329200,0.402021,0.003300,0.869800
600,0.427000,0.343541,0.003300,0.879200
700,0.333000,0.311375,0.003300,0.875200
800,0.331400,0.361965,0.003300,0.875600
900,0.410800,0.304343,0.003300,0.874000
1000,0.555000,0.318875,0.003300,0.880200


In [13]:
#print("\nRoBERTa Accuracy:", roberta_results["accuracy"])
print("BERT Accuracy:", bert_results["eval_accuracy"])

TypeError: 'NoneType' object is not subscriptable